<a href="https://colab.research.google.com/github/garry-p12/QwenVL2/blob/main/QwenVL2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-j5thg6uy
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-j5thg6uy
  Resolved https://github.com/huggingface/transformers to commit 33868a057c02f0368ba63bd1edb746be38fe3d90
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
pip install qwen-vl-utils

In [3]:
pip install ms-vlmeval

In [4]:
pip install datasets

In [5]:
from datasets import load_dataset

ds = load_dataset("HuggingFaceM4/MMBench_dev")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
# from datasets import load_dataset
# from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
# import torch
# from qwen_vl_utils import process_vision_info

# # Load the dataset
# ds = load_dataset("weijiezz/mmbench_en_dev_split_test")

# # Load the model on available device(s)
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model = Qwen2VLForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen2-VL-2B-Instruct", torch_dtype="auto", device_map="auto"
# ).to(device)

# # Recommended processor settings for multi-image scenarios
# min_pixels = 256 * 28 * 28
# max_pixels = 1280 * 28 * 28
# processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)

# # Set the number of samples (questions) to be processed for the LLM
# num_samples = len(ds['dev'])  # Use all samples in the 'dev' split

# # Iterate over each sample (image-question pair) and process them individually
# for i in range(num_samples):
#     sample = ds['dev'][i]  # Access each sample from the 'dev' split

#     # Extract image
#     img = sample['image']

#     # Build a text prompt combining question, hint, and options (A, B, C, D)
#     question = sample['question']
#     hint = sample.get('hint', '')  # Some hints may be null
#     options = f"A: {sample['A']}, B: {sample['B']}, C: {sample['C']}, D: {sample['D']}"

#     # Prepare prompt
#     prompt = f"Question: {question}\nHint: {hint}\nOptions: {options}"

#     # Create messages for the LLM containing the image and corresponding text prompt
#     messages = [{
#         "role": "user",
#         "content": [
#             {"type": "image", "image": img},
#             {"type": "text", "text": prompt},
#             {"type": "text", "text": "Describe this image and answer the question."}
#         ]
#     }]

#     # Prepare the input for the model
#     text = processor.apply_chat_template(
#         messages, tokenize=False, add_generation_prompt=True
#     )
#     image_inputs, video_inputs = process_vision_info(messages)

#     # Ensure all inputs are on the same device
#     inputs = processor(
#         text=[text],
#         images=image_inputs,
#         videos=video_inputs,
#         padding=True,
#         return_tensors="pt",
#     ).to(device)  # Move the inputs to the correct device

#     # Inference: Generate the output
#     generated_ids = model.generate(**inputs, max_new_tokens=128)
#     generated_ids_trimmed = [
#         out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
#     ]

#     # Decode the output to generate text
#     output_text = processor.batch_decode(
#         generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
#     )

#     # Print the generated descriptions and answers for the current image and question
#     print(f"Generated description and answer for sample {i+1}:\n{output_text[0]}\n")



In [7]:
pip install vllm

In [8]:
from datasets import load_dataset
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
import torch
from qwen_vl_utils import process_vision_info

ds = load_dataset("weijiezz/mmbench_en_dev_split_test")

# Load the model on available device(s)
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct", torch_dtype="auto"
).to(device)  # Ensuring the model is loaded to cuda:0

min_pixels = 256 * 28 * 28
max_pixels = 1280 * 28 * 28
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)

num_samples = 5
# Extract data from the dataset (including image and text fields)
image_data = []
text_prompts = []

for i in range(num_samples):
    sample = ds['dev'][i]  # Access each sample from the 'dev' split

    # Extract image
    img = sample['image']
    image_data.append(img)

    # Build a text prompt combining question, hint, and options (A, B, C, D)
    question = sample['question']
    hint = sample.get('hint', '')  # Some hints may be null
    options = f"A: {sample['A']}, B: {sample['B']}, C: {sample['C']}, D: {sample['D']}"

    # Prepare prompt
    prompt = f"Question: {question}\nHint: {hint}\nOptions: {options}"
    text_prompts.append(prompt)

# Create messages list for the LLM containing multiple images and corresponding text prompts
messages = [{"role": "user", "content": []}]

# Add each image and corresponding prompt to the messages
for img, prompt in zip(image_data, text_prompts):
    messages[0]["content"].append({"type": "image", "image": img})
    messages[0]["content"].append({"type": "text", "text": prompt})

messages[0]["content"].append({"type": "text", "text": "Describe these images and answer the questions."})

text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)

# Ensure all inputs are on the same device (cuda:0)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
).to(device)  # Move the inputs to cuda:0

generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]

output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)

for i, text in enumerate(output_text):
    print(f"Generated description and answer for image set {i+1}:\n{text}")


`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Generated description and answer for image set 1:
The image depicts a person painting a wooden board with a brush. The profession of the person in the picture is most likely a carpenter or a painter. The cityscape in the background suggests that the location is in China, as the Great Wild Goose Pagoda is a well-known landmark in that country. Therefore, the correct answer is C: carpenter.


In [9]:
pip install vllm

In [24]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
torch.backends.cuda.matmul.allow_tf32 = False

In [36]:
from datasets import load_dataset
from transformers import AutoProcessor
import torch
from vllm import LLM, SamplingParams
from qwen_vl_utils import process_vision_info


ds = load_dataset("weijiezz/mmbench_en_dev_split_test")

min_pixels = 256 * 28 * 28
max_pixels = 1280 * 28 * 28
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)

llm = LLM(
    model="Qwen/Qwen2-VL-2B-Instruct",
    limit_mm_per_prompt={"image": 10, "video": 10},
    dtype="float16",
    tensor_parallel_size=1,
    enforce_eager=True,
)

num_samples = len(ds['dev'])  # Use all samples in the 'dev' split

for i in range(num_samples):
    sample = ds['dev'][i]
    img = sample['image']

    # Build a text prompt combining question, hint, and options (A, B, C, D)
    question = sample['question']
    hint = sample.get('hint', '')  # Some hints may be null
    options = f"A: {sample['A']}, B: {sample['B']}, C: {sample['C']}, D: {sample['D']}"

    prompt = f"Question: {question}\nHint: {hint}\nOptions: {options}"

    messages = [{
        "role": "user",
        "content": [
            {"type": "image", "image": img},
            {"type": "text", "text": prompt},
            {"type": "text", "text": "Describe this image and answer the question."}
        ]
    }]

    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    image_inputs, video_inputs = process_vision_info(messages)

    # Combine the prompt with image and process it
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt"
    )

    inputs = inputs.to("cuda")

    sampling_params = SamplingParams(max_tokens=128)

    # Run inference using vLLM's engine
    output = llm.generate([text], sampling_params)

    print(f"Generated description and answer for sample {i+1}:\n{output[0].outputs[0].text}\n")


WARNING 11-03 20:03:54 config.py:1668] Casting torch.bfloat16 to torch.float16.
WARNING 11-03 20:03:54 config.py:395] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 11-03 20:03:54 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='Qwen/Qwen2-VL-2B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2-VL-2B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=Observa

RuntimeError: Offset increment outside graph capture encountered unexpectedly.

In [32]:
pip install ray

In [34]:
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B-Instruct")

# Pass the default decoding hyperparameters of Qwen2.5-7B-Instruct
# max_tokens is for the maximum length for generation.
sampling_params = SamplingParams(temperature=0.7, top_p=0.8, repetition_penalty=1.05, max_tokens=512)

# Input the model name or path. Can be GPTQ or AWQ models.
llm = LLM(
    model="Qwen/Qwen2.5-7B-Instruct-AWQ",
    quantization="awq",
    dtype="float16",
    max_model_len=4096,  # Reduced from 32768 to save memory
    gpu_memory_utilization=0.7,  # Adjusted GPU memory usage, reduced to 70% from 90%
    enforce_eager=True
)

prompt = "Tell me something about large language models."
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

outputs = llm.generate([text], sampling_params)

for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

WARNING 11-03 19:45:05 config.py:321] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
WARNING 11-03 19:45:05 config.py:395] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 11-03 19:45:05 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='Qwen/Qwen2.5-7B-Instruct-AWQ', speculative_config=None, tokenizer='Qwen/Qwen2.5-7B-Instruct-AWQ', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=awq, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 11-03 19:45:34 model_runner.py:1067] Loading model weights took 5.1966 GB
INFO 11-03 19:45:35 model_runner_base.py:120] Writing input of failed execution to /tmp/err_execute_model_input_20241103-194535.pkl...
INFO 11-03 19:45:37 model_runner_base.py:149] Completed writing input of failed execution to /tmp/err_execute_model_input_20241103-194535.pkl.


RuntimeError: Error in model execution (input dumped to /tmp/err_execute_model_input_20241103-194535.pkl): Offset increment outside graph capture encountered unexpectedly.